In [1]:
import numpy as np

In [64]:
def concat_popped_to_previous(A, B, C):
    N, M = A.shape
    A = A.astype(object)  # ensure string operations behave correctly

    A_new = A.copy()

    # Get the elements to be popped
    popped = A[np.arange(N), B]

    # Concatenate to the previous element in each row
    A_new[np.arange(N), B - 1] = "(" + A_new[np.arange(N), B - 1] + C[np.arange(N)] + popped + ")"

    # Mask to remove B[i]-th column from each row
    col_indices = np.arange(M)
    mask = col_indices[None, :] != B[:, None]
    result = A_new[mask].reshape(N, M - 1)

    return result

def one_step_reduction(A, B, C):
    N, M = A.shape
    A = A.astype(object)

    A_new = A.copy()

    # Indexing
    idx = np.arange(N)

    # Get elements to combine
    popped = A[idx, B]
    prev = A[idx, B - 1]
    ops = C[idx]

    # Convert to integers
    x = prev.astype(int)
    y = popped.astype(int)

    # Compute result vectorized using np.where
    reduced = np.where(ops == "+", (x + y) % 10, (x - y) % 10).astype(str)

    # Write back to A_new
    A_new[idx, B - 1] = reduced

    # Remove the popped column
    col_indices = np.arange(M)
    mask = col_indices[None, :] != B[:, None]
    result = A_new[mask].reshape(N, M - 1)

    return result

def concat_op_var(A,C):
    N = A.shape[0]
    A = A.astype(object)  # ensure string operations behave correctly
    A_new = A.copy()
    A_new[np.arange(N), 0] = A_new[np.arange(N), 0] + C[np.arange(N)] + A_new[np.arange(N), 1]
    return A_new[:, :1]

def generate_constrained_sequences(batch_size, T, high=None):
    if high is None:
        high = T
    elif T > high:
        raise ValueError("T cannot be greater than high")

    # Shape: (1, T) → [[high, high-1, ..., high-T+1]]
    upper_bounds = high - np.arange(T)

    # Uniform samples in [0, 1), shape: (batch_size, T)
    random_floats = np.random.rand(batch_size, T)

    # Scale to [1, upper_bound] per timestep
    # floor(random * bound) ∈ [0, bound-1], then +1 ∈ [1, bound]
    samples = np.floor(random_floats * upper_bounds).astype(int) + 1

    return samples

def sample_arith_exp(num_samples, num_variables):
    variables = np.random.randint(low=0, high=9, size=(num_samples, num_variables)).astype(str)
    ops = np.random.choice(["+", "-"], size=(num_samples, num_variables-1))
    orders = generate_constrained_sequences(num_samples, num_variables-1)
    one_step_result = one_step_reduction(variables, orders[:, 0], ops[:, 0])
    variables = concat_popped_to_previous(variables, orders[:, 0], ops[:, 0])
    for t in range(1,num_variables-2):
        one_step_result = concat_popped_to_previous(one_step_result, orders[:, t], ops[:, t])
        variables = concat_popped_to_previous(variables, orders[:, t], ops[:, t])
    variables = concat_op_var(variables, ops[:,num_variables-2])
    one_step_result = concat_op_var(one_step_result, ops[:,num_variables-2])
    return variables, one_step_result

In [65]:
sample_arith_exp(5, 4)

(array([['7-((7+1)-0)'],
        ['(1+(5-5))+3'],
        ['((6+2)+0)+5'],
        ['(4+3)+(7+0)'],
        ['(8-0)+(7-0)']], dtype=object),
 array([['7-(8-0)'],
        ['(1+0)+3'],
        ['(8+0)+5'],
        ['7+(7+0)'],
        ['8+(7-0)']], dtype=object))